<a href="https://colab.research.google.com/github/alejandromunizsiques/Ejercicios-Coding-Dojo/blob/main/Ejercicio_sobre_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importacion de  liberias


In [1]:
import pandas as pd
from sqlalchemy import create_engine

# Montaje drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Montaje datasets

In [4]:
school = pd.read_csv("/content/drive/MyDrive/Data Science Coding Dojo/Machine Learning Avanzado/school_details.csv")
school.head(2)

,districtname,establishmentcode,establishmentname,brcname,crcname,schoollevel,managementname,localityhabitation,subdivisionname,constituencyname,revenueblockname,civicbodyname,wardname,postofficename,phoneno,faxno,emailid,website,locatedinruralorurban,disecode,establisheddate,districthqdistance_in_km,crcdistance_in_km,bacdistance_in_km,isrecognized,recognizedby,contactpersonname,lowestclass,highestclass,boardname
0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,Central Pendam,Central Pendam,Sajong,Upper Pendam,NaN,NaN,NaN,NaN,Urban Area,NaN,28126,58.0,8.0,16.0,Recognized,HRDD,NaN,Class 1,Class 5,NaN
1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,Central Pendam,Central Pendam,Karmithang,Upper Pendam,3592263812,NaN,NaN,NaN,Urban Area,800401,15342,50.0,0.0,6.0,Recognized,HRDD,NaN,Class 1,Class 12,C.B.S.E.


In [5]:
facility = pd.read_csv('/content/drive/MyDrive/Data Science Coding Dojo/Machine Learning Avanzado/school_facility_details.csv')
facility.head(2)

,establishmentcode,establishmentname,islibraryavailable,availablebookscount,isfireextgavailable,extinguishercount,hoiroomavailable,isstaffroomavailable,noofstaffroom,islockerforstaffavailable,issportsstoreroomavailable
0,S1034,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,Not Available,0,Available,1,Available,Not Available,0,Not Available,Not Available
1,S0516,10TH MILE GOVERNMENT PRIMARY SCHOOL,Not Available,0,Not Available,0,Available,Available,1,Not Available,Not Available


# Creacion de Engine

In [6]:
# create sqlite engine
engine = create_engine('sqlite:///school.db', echo=True)
# create connection to engine
conn = engine.connect()

# Creacion data sql

In [7]:
sqlite_table = "school_details"
school.to_sql(sqlite_table, conn, if_exists='fail')

sqlite_table2 = "school_facility_details"
facility.to_sql(sqlite_table2, conn, if_exists='fail')

2021-11-05 22:55:34,948 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("school_details")
2021-11-05 22:55:34,950 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-05 22:55:34,957 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("school_details")
2021-11-05 22:55:34,959 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-05 22:55:34,973 INFO sqlalchemy.engine.Engine 
CREATE TABLE school_details (
	"index" BIGINT, 
	districtname TEXT, 
	establishmentcode TEXT, 
	establishmentname TEXT, 
	brcname TEXT, 
	crcname TEXT, 
	schoollevel TEXT, 
	managementname TEXT, 
	localityhabitation TEXT, 
	subdivisionname TEXT, 
	constituencyname TEXT, 
	revenueblockname TEXT, 
	civicbodyname TEXT, 
	wardname TEXT, 
	postofficename TEXT, 
	phoneno TEXT, 
	faxno TEXT, 
	emailid TEXT, 
	website TEXT, 
	locatedinruralorurban TEXT, 
	disecode TEXT, 
	establisheddate TEXT, 
	districthqdistance_in_km FLOAT, 
	crcdistance_in_km FLOAT, 
	bacdistance_in_km FLOAT, 
	isrecognized TEXT, 
	recognizedby TEXT, 

# Preguntas

¿Cuál es el BACDistance promedio (en km)? Renombra la columna resultante como "avg_bac_distancia."

In [8]:
# Write your SQL query in this string
sql = """
SELECT AVG("BACDistance (in Km)") AS avg_bac_distance
FROM school_details
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 22:56:44,681 INFO sqlalchemy.engine.Engine 
SELECT AVG("BACDistance (in Km)") AS avg_bac_distance
FROM school_details

2021-11-05 22:56:44,684 INFO sqlalchemy.engine.Engine [raw sql] ()


,avg_bac_distance
0,0.0


¿Cuál es el BACDistance promedio (en km) según nivel educativo del colegio?  Ordena los resultados de promedio mayor a promedio menor.

In [9]:
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 22:58:08,196 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
ORDER BY avg_bac_distance DESC

2021-11-05 22:58:08,203 INFO sqlalchemy.engine.Engine [raw sql] ()


,schoollevel,avg_bac_distance
0,None,0.0
1,JHS,0.0
2,LPS,0.0
3,PS,0.0
4,SS,0.0


Repite la consulta anterior, pero solo muestra los resultados donde la distancia promedio sea menor a 10 km.

In [10]:
# Write your SQL query in this string
sql = """
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 22:59:32,538 INFO sqlalchemy.engine.Engine 
SELECT SchoolLevel, AVG("BACDistance (in Km)") as avg_bac_distance
FROM school_details
GROUP BY SchoolLevel
HAVING avg_bac_distance >= 10
ORDER BY avg_bac_distance DESC

2021-11-05 22:59:32,542 INFO sqlalchemy.engine.Engine [raw sql] ()


,schoollevel,avg_bac_distance


Une las dos tablas (detalles de colegios y detalles de instalaciones de los colegios). Comienza mostrando todas las columnas, pero limita las filas resultantes a solo 2.

In [11]:
# Write your SQL query in this string
sql = """
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 23:00:01,377 INFO sqlalchemy.engine.Engine 
SELECT *
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
LIMIT 2

2021-11-05 23:00:01,380 INFO sqlalchemy.engine.Engine [raw sql] ()


,index,districtname,establishmentcode,establishmentname,brcname,crcname,schoollevel,managementname,localityhabitation,subdivisionname,constituencyname,revenueblockname,civicbodyname,wardname,postofficename,phoneno,faxno,emailid,website,locatedinruralorurban,disecode,establisheddate,districthqdistance_in_km,crcdistance_in_km,bacdistance_in_km,isrecognized,recognizedby,contactpersonname,lowestclass,highestclass,boardname,index,establishmentcode,establishmentname,islibraryavailable,availablebookscount,isfireextgavailable,extinguishercount,hoiroomavailable,isstaffroomavailable,noofstaffroom,islockerforstaffavailable,issportsstoreroomavailable
0,0,East,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Duga Brc,Central Pendam Government Senior Secondary School,PS,State Govt.,BUDANG,Gangtok,Rhenock,Central Pendam,Central Pendam,Sajong,Upper Pendam,None,None,None,None,Urban Area,None,28126,58.0,8.0,16.0,Recognized,HRDD,None,Class 1,Class 5,None,121,S0169,BUDANG GOVERNMENT PRIMARY SCHOOL (E),Available,100,Available,2,Available,Available,1,Not Available,Not Available
1,1,East,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Duga Brc,Central Pendam Government Senior Secondary School,SSS,State Govt.,CENTRAL PENDAM,Gangtok,West Pandam,Central Pendam,Central Pendam,Karmithang,Upper Pendam,3592263812,None,None,None,Urban Area,800401,15342,50.0,0.0,6.0,Recognized,HRDD,None,Class 1,Class 12,C.B.S.E.,137,S0099,CENTRAL PENDAM GOVERNMENT SENIOR SECONDARY SCHOOL,Available,0,Available,5,Available,Available,2,Not Available,Available


Ahora, solo selecciona colegios donde el nivel de este sea Escuela Primaria y que no tengan biblioteca.

In [12]:
# Write your SQL query in this string
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 23:00:36,627 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'

2021-11-05 23:00:36,633 INFO sqlalchemy.engine.Engine [raw sql] ()


,establishmentname,schoollevel,islibraryavailable
0,EASWARAMMA SAI GURUKOOL ACCADEMY,PS,Not Available
1,NEW GARDEN BOARDING SCHOOL (SAJONG),PS,Not Available
2,"SARASWATI SHISHU VIDHYALAYA, BUDANG",PS,Not Available
3,SAWNEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,UPPER PACHAK GOVERNMENT PRIMARY SCHOOL,PS,Not Available


Toma la consulta anterior, pero ordena los resultados en orden alfabético por nombre.

In [13]:
sql = """
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName
"""

# Read the sql query into pandas to view the results
df = pd.read_sql_query(sql, engine)
df.head()

2021-11-05 23:01:22,206 INFO sqlalchemy.engine.Engine 
SELECT s.EstablishmentName, s.SchoolLevel, f.IsLibraryAvailable
FROM school_details s
  JOIN school_facility_details f
  ON f.EstablishmentCode = s.EstablishmentCode
WHERE s.SchoolLevel = 'PS' AND f.IsLibraryAvailable = 'Not Available'
ORDER BY s.EstablishmentName

2021-11-05 23:01:22,209 INFO sqlalchemy.engine.Engine [raw sql] ()


,establishmentname,schoollevel,islibraryavailable
0,(I.E.C) INDIAN EVANGALISTIC CRUSADE MISSION PR...,PS,Not Available
1,10TH MILE GOVERNMENT PRIMARY SCHOOL,PS,Not Available
2,AAPGAZI GOVERNMENT PRIMARY SCHOOL,PS,Not Available
3,AARUBOTEY GOVERNMENT PRIMARY SCHOOL,PS,Not Available
4,AHO-YANGTAM GOVERNMENT PRIMARY SCHOOL,PS,Not Available
